# 프로젝트 설명

In [1]:
import sys
sys.path.append("..")


In [6]:
from src.common import *
project_name = "dual_attention"
createfolder("./data")
createfolder("./model")
createfolder("./result")

# Data load

In [16]:
from src.rnn.datapreparation import *

In [15]:
a, b = data2series(data=)

TypeError: data2series() missing 5 required positional arguments: 'data', 'history_length', 'history_var', 'future_length', and 'future_var'